Cross Validation 4

size =  20247

test size 4k+

epoch-5

apply sigmoid (prediction probabilities are logits)

groups-9

target_list = ['Camera', 'Location', 'Microphone', 'Contacts', 'Storage', 'Phone', 'SMS', 'Call_Log', 'Calendar']

threshold-tuning = yes

df_2k = df[(df['Rating'] >= 4.5) & (df['Maximum_Installs'] >= 20000)]

accuracy score : F1 score (micro) and ROC_AUC score

## 1. Setup

In [1]:
import tensorflow as tf

# Get the GPU device name.
device_name = tf.test.gpu_device_name()

# The device name should look like the following:
if device_name == '/device:GPU:0':
    print('Found GPU at: {}'.format(device_name))
else:
    raise SystemError('GPU device not found')

Found GPU at: /device:GPU:0


In [2]:
print(tf.__version__)

2.8.0


In [3]:
import torch

# If there's a GPU available...
if torch.cuda.is_available():    

    # Tell PyTorch to use the GPU.    
    device = torch.device("cuda")

    print('There are %d GPU(s) available.' % torch.cuda.device_count())

    print('We will use the GPU:', torch.cuda.get_device_name(0))

# If not...
else:
    print('No GPU available, using the CPU instead.')
    device = torch.device("cpu")

There are 1 GPU(s) available.
We will use the GPU: Tesla K80


## 2. Load Dataset

In [4]:
## import libraries

import itertools
import numpy as np
import matplotlib.pyplot as plt

import pandas as pd
import numpy as np

from sklearn import preprocessing
%matplotlib inline

In [5]:
## uploading csv files on drive (to avoid uploading on colab in every session)

from google.colab import drive
drive.mount("/content/drive/")

## drive path
train_path = "/content/drive/MyDrive/MetadataCSV/data_20247/CV_df_train_4.csv"
val_path = "/content/drive/MyDrive/MetadataCSV/data_20247/CV_df_val_4.csv"
test_path = "/content/drive/MyDrive/MetadataCSV/test_dataset.csv"

Mounted at /content/drive/


In [6]:
df_train = pd.read_csv(train_path) 
df_val = pd.read_csv(val_path)
df_test = pd.read_csv(test_path)

print(df_train.shape)
print(df_val.shape)
print(df_test.shape)

(16197, 23)
(4050, 23)
(4624, 23)


In [7]:
df_train.head(2)

,App_Name,App_Id,Category,Rating,Maximum_Installs,Editors_Choice,Description,Privacy_Policy,Sensors,Camera,...,Contacts,SMS,Storage,Phone,Get_Accounts,Call_Log,desc_length,Clean_Description,clean_desc_length,token_length
0,Fortune Teller Khmer,com.khemarasoft.fortuneteller,Lifestyle,4.5,50284,False,Want to find out your future? This app is a mu...,http://khemarasoft.com/app-policy,0,0,...,0,0,0,0,0,0,370,want to find out your future? this app is a mu...,365,76
1,Mrsool | مرسول,com.mrsool,Shopping,4.5,6450386,False,Mrsool is one of the largest delivery platform...,https://s.mrsool.co/privacy_policy.html,0,1,...,0,0,1,1,1,0,1356,mrsool is one of the largest delivery platform...,1310,267


In [8]:
target_list = ['Camera', 'Location', 'Microphone', 'Contacts', 'Storage', 'Phone', 'SMS', 'Call_Log', 'Calendar']

In [9]:
# getting number of nonzeros in each column
df_train[target_list].astype(bool).sum(axis=0)

Camera        2469
Location      2459
Microphone    1464
Contacts      1622
Storage       8226
Phone         2392
SMS             54
Call_Log        32
Calendar       279
dtype: int64

In [10]:
df_val[target_list].astype(bool).sum(axis=0)

Camera         629
Location       621
Microphone     386
Contacts       428
Storage       2027
Phone          607
SMS             17
Call_Log        11
Calendar        73
dtype: int64

In [11]:
df_test[target_list].astype(bool).sum(axis=0)

Camera         745
Location       716
Microphone     436
Contacts       500
Storage       2402
Phone          652
SMS             11
Call_Log         6
Calendar        90
dtype: int64

## 3. Data Preprocess

#### 3.1 Clean Text

In [12]:
import nltk

nltk.download("stopwords")
nltk.download('punkt')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [13]:
from nltk.corpus import stopwords

stop = set(stopwords.words("english"))

def remove_stopwords(text):
    text = [word.lower() for word in text.split() if word.lower() not in stop]

    return " ".join(text)

In [14]:
df_train["Clean_Description"] = df_train["Clean_Description"].map(remove_stopwords)
df_val["Clean_Description"] = df_val["Clean_Description"].map(remove_stopwords)
df_test["Clean_Description"] = df_test["Clean_Description"].map(remove_stopwords)

In [15]:
import string

def remove_punct(text):
    table = str.maketrans("", "", string.punctuation)
    return text.translate(table)

In [16]:
df_train["Clean_Description"] = df_train["Clean_Description"].map(lambda x: remove_punct(x))
df_val["Clean_Description"] = df_val["Clean_Description"].map(lambda x: remove_punct(x))
df_test["Clean_Description"] = df_test["Clean_Description"].map(lambda x: remove_punct(x))

#### 3.2 Create Corpus

In [17]:
from keras.layers import *

In [18]:
import os
import sys
import numpy as np
import keras
from keras.preprocessing.text import Tokenizer
from tensorflow.keras.utils import to_categorical
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Activation, Conv2D, Input, Embedding, Reshape, MaxPool2D, Concatenate, Flatten, Dropout, Dense, Conv1D
from keras.layers import MaxPool1D
from keras.models import Model
from keras.callbacks import ModelCheckpoint
from tensorflow.keras.optimizers import Adam

In [19]:
from nltk.tokenize import word_tokenize
import nltk


def create_corpus_tk(df):
    corpus = []
    for text in df["Clean_Description"]:
        words = [word.lower() for word in word_tokenize(text)]
        corpus.append(words)
    return corpus

In [20]:
df_T = df_train.append(df_val)
df_T.reset_index(drop=True,inplace=True)

In [21]:
corpus = create_corpus_tk(df_T)

In [22]:
num_words = len(corpus)
print(num_words)

20247


In [23]:
corpus[0]

['want',
 'find',
 'future',
 'app',
 'must',
 'like',
 'read',
 'fortune',
 'fortune',
 'teller',
 'khmer',
 'collection',
 'khmer',
 'language',
 'simple',
 'way',
 'laughs',
 'fun',
 'simple',
 'use',
 'input',
 'phone',
 'number',
 'name',
 'birthday',
 'etc',
 'application',
 'intended',
 'entertainment',
 'purpose',
 'actually',
 'capable',
 'telling',
 'future']

##4. Train Validation Split

In [ ]:
# split the data into a training set and a validation set

In [24]:
train_inputs = df_train["Clean_Description"]
validation_inputs = df_val["Clean_Description"]
train_labels = df_train[target_list]
validation_labels = df_val[target_list]

## 5. Tokenization

In [25]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

tokenizer = Tokenizer(num_words=num_words)
tokenizer.fit_on_texts(train_inputs)

In [26]:
MAX_SEQUENCE_LENGTH = 600

In [27]:
train_sequences = tokenizer.texts_to_sequences(train_inputs)

In [28]:
train_inputs

0        want find future app must like read fortune fo...
1        mrsool one largest delivery platforms kingdom ...
2        hombi social app managing apartment buildings ...
3        oos skin playerpro free color accents pink blu...
4        lokseva academy online platform managing data ...
                               ...                        
16192    time workouts plans hybrid interval timer app ...
16193    ever twitch play loudest gachi want express lo...
16194      synonyms relationships entertainment birds v...
16195    colorful lion theme gives best keyboard backgr...
16196    keep track weekly k run results parkrunner ana...
Name: Clean_Description, Length: 16197, dtype: object

In [29]:
train_sequences

[[37,
  28,
  472,
  1,
  391,
  19,
  148,
  2048,
  2048,
  5590,
  1082,
  164,
  1082,
  45,
  52,
  69,
  15869,
  113,
  52,
  6,
  339,
  7,
  116,
  263,
  993,
  60,
  10,
  1397,
  999,
  1039,
  2342,
  5365,
  3601,
  472],
 [12822,
  17,
  1815,
  1087,
  1957,
  2598,
  12822,
  173,
  2192,
  86,
  4277,
  1766,
  89,
  2442,
  933,
  742,
  1087,
  39,
  151,
  23,
  2873,
  1,
  4499,
  487,
  495,
  1177,
  3050,
  663,
  1836,
  1544,
  2598,
  2873,
  3211,
  289,
  5366,
  4745,
  3375,
  6299,
  722,
  807,
  2459,
  12822,
  1087,
  1,
  2629,
  1550,
  12822,
  19,
  4984,
  225,
  1398,
  934,
  316,
  2629,
  584,
  3050,
  16,
  2629,
  407,
  247,
  1286,
  4317,
  2330,
  2221,
  140,
  4091,
  931,
  6300,
  874,
  12822,
  547,
  352,
  12822,
  2032,
  4499,
  487,
  121,
  2288,
  37,
  352,
  998,
  6,
  12822,
  4447,
  8134,
  418,
  316,
  600,
  415,
  808,
  1550,
  316,
  87,
  316,
  167,
  1481,
  316,
  1836,
  3050,
  1177,
  10557,
  225,
  

In [30]:
train_padded = pad_sequences(
    train_sequences, maxlen=MAX_SEQUENCE_LENGTH, truncating="post", padding="post"
)

In [31]:
train_padded

array([[   37,    28,   472, ...,     0,     0,     0],
       [12822,    17,  1815, ...,     0,     0,     0],
       [19446,   157,     1, ...,     0,     0,     0],
       ...,
       [ 3545,  2463,   999, ...,     0,     0,     0],
       [  212,   734,     3, ...,     0,     0,     0],
       [  119,   191,   743, ...,     0,     0,     0]], dtype=int32)

In [32]:
validation_sequences = tokenizer.texts_to_sequences(validation_inputs)
validation_padded = pad_sequences(
    validation_sequences, maxlen=MAX_SEQUENCE_LENGTH, padding="post", truncating="post"
)

In [33]:
validation_padded

array([[15198,   801,   215, ...,     0,     0,     0],
       [ 2935,   159,   127, ...,     0,     0,     0],
       [ 5275,     1,   102, ...,     0,     0,     0],
       ...,
       [  800,     9,     4, ...,     0,     0,     0],
       [  324,   143,  1054, ...,     0,     0,     0],
       [  464,  2946,   627, ...,     0,     0,     0]], dtype=int32)

In [34]:
print(df_train.Clean_Description[0])
print(train_sequences[0])

want find future app must like read fortune fortune teller khmer collection khmer language simple way laughs fun simple use input phone number name birthday etc application intended entertainment purpose actually capable telling future
[37, 28, 472, 1, 391, 19, 148, 2048, 2048, 5590, 1082, 164, 1082, 45, 52, 69, 15869, 113, 52, 6, 339, 7, 116, 263, 993, 60, 10, 1397, 999, 1039, 2342, 5365, 3601, 472]


In [35]:
word_index = tokenizer.word_index
print("Number of unique words:", len(word_index))

Number of unique words: 66449


In [36]:
word_index

{'app': 1,
 'keyboard': 2,
 'theme': 3,
 'free': 4,
 'wallpaper': 5,
 'use': 6,
 'phone': 7,
 'wallpapers': 8,
 'new': 9,
 'application': 10,
 'features': 11,
 'get': 12,
 'time': 13,
 'make': 14,
 'android': 15,
 'also': 16,
 'one': 17,
 'launcher': 18,
 'like': 19,
 'download': 20,
 'hd': 21,
 'screen': 22,
 'best': 23,
 'english': 24,
 'learn': 25,
 'easy': 26,
 'us': 27,
 'find': 28,
 'love': 29,
 'themes': 30,
 'live': 31,
 'help': 32,
 'share': 33,
 'mobile': 34,
 'please': 35,
 'set': 36,
 'want': 37,
 'support': 38,
 'apps': 39,
 'words': 40,
 'home': 41,
 'device': 42,
 'day': 43,
 'need': 44,
 'language': 45,
 'enjoy': 46,
 'images': 47,
 'using': 48,
 'available': 49,
 'different': 50,
 'many': 51,
 'simple': 52,
 'note': 53,
 'beautiful': 54,
 'information': 55,
 'add': 56,
 'apply': 57,
 'daily': 58,
 'learning': 59,
 'etc': 60,
 'video': 61,
 'text': 62,
 'you': 63,
 'background': 64,
 'access': 65,
 'every': 66,
 'friends': 67,
 'easily': 68,
 'way': 69,
 'install': 70,


In [37]:
word_index["reason"]

2507

In [38]:
print(validation_sequences[0])

[15198, 801, 215, 74, 120, 801, 52, 73, 69, 489, 418, 361, 297, 120, 26, 853, 285, 213, 16910, 452, 817, 132, 659, 41, 3255, 3929, 419, 307, 285, 44, 667, 801, 16910, 285, 1, 1261, 120, 801, 213, 684, 16, 1238, 255, 3018, 1276, 3179, 188, 626, 307, 285, 19, 1782, 3026, 18873, 3018, 213, 474, 425, 532, 1192, 3179, 532, 694, 84, 1327, 67, 120, 2545, 439, 2455, 9262, 1281, 303, 196, 74, 120, 801, 26, 65, 16910, 297, 297, 2098, 5371, 63, 1472, 16910, 2455, 487, 52, 956, 69, 145, 43, 43, 1, 270, 74, 2545, 785, 297, 518, 2545, 783, 785, 43, 1446, 16, 50, 297, 297, 1261, 812, 120, 801, 1018, 196, 817, 49, 17, 74, 507, 6, 1, 255, 3179, 1019, 120, 801, 43, 68, 198, 4899, 1134, 51, 3095, 447, 3879, 2835, 472, 1857, 9, 196, 692, 1160, 718, 5985, 585]


In [39]:
word_index["listen"]

300

## 6. Create the Embedding dictionary

In [40]:
embedding_dict = {}
with open("/content/drive/MyDrive/glove6B/glove.6B.300d.txt", "r") as f:
    for line in f:
        values = line.split()
        word = values[0]
        vectors = np.asarray(values[1:], "float32")
        embedding_dict[word] = vectors
f.close()

In [41]:
embedding_dict

Output hidden; open in https://colab.research.google.com to view.

In [42]:
embedding_dim = 300

In [43]:
num_words = len(word_index) + 1
embedding_matrix = np.zeros((num_words, embedding_dim))

for word, i in word_index.items():
    if i < num_words:
        emb_vec = embedding_dict.get(word)
        if emb_vec is not None:
            embedding_matrix[i] = emb_vec

In [44]:
embedding_matrix

array([[ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [-0.73247999, -0.078309  ,  0.018521  , ...,  0.21988   ,
        -0.13121   ,  0.043819  ],
       [-0.51651001,  0.090911  , -0.15025   , ...,  0.17309999,
         0.094735  , -0.62935001],
       ...,
       [ 0.37533   , -0.32378   ,  0.49807   , ..., -0.27970001,
         0.27811   ,  0.10715   ],
       [-0.16889   , -0.16912   ,  0.47323   , ..., -0.091788  ,
        -0.14264999, -0.68147999],
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ]])

In [45]:
word_index["reason"]

2507

In [46]:
embedding_dict.get("reason")

array([ 1.3197e-01, -1.2591e-01,  4.3864e-02,  3.6321e-02,  9.6646e-02,
       -1.3829e-01,  3.8637e-01,  7.6962e-02, -1.1306e-01, -1.6083e+00,
        2.0062e-02, -5.2665e-02, -1.6597e-01,  1.2171e-01,  2.8945e-01,
       -1.7289e-01,  5.3035e-02, -2.7842e-01,  8.2376e-02, -1.1980e-02,
        3.7228e-02,  2.1867e-01,  1.5267e-01, -8.4361e-02, -3.1292e-01,
       -3.2093e-02,  2.0281e-01, -3.5910e-01,  1.6873e-02, -2.2996e-01,
        2.6044e-02,  3.5910e-01, -3.2431e-01, -5.4194e-01, -9.7742e-01,
        3.9198e-02, -1.7794e-01,  7.4200e-02, -4.1251e-02, -7.8917e-02,
        2.0646e-01, -6.6538e-02,  6.7401e-02,  1.4965e-01,  5.9107e-02,
       -3.7585e-02, -3.4672e-02,  5.5291e-02, -8.5636e-02,  9.1743e-02,
        4.9125e-01,  7.5606e-03, -3.0860e-01,  5.8902e-04, -8.6975e-02,
        3.9904e-01, -1.2695e-01,  2.2471e-01,  2.3658e-01,  3.0489e-01,
       -6.7363e-02,  3.5839e-01,  4.9703e-01,  4.1895e-01, -3.8494e-01,
       -2.6257e-01,  1.6049e-01, -1.0992e-01,  2.7477e-02,  1.49

In [47]:
(embedding_matrix[2363] == embedding_dict.get("reason")).all()

False

In [48]:
print(train_padded.shape)
print(train_labels.shape)

(16197, 600)
(16197, 9)


In [49]:
print(validation_padded.shape)
print(validation_labels.shape)

(4050, 600)
(4050, 9)


## 7. Build CNN Model

In [50]:
num_permissions = 9 #11
drop = 0.2

batch_size = 32
max_train_epochs = 300
validation_split = 6

early_stopping_patience = 16
early_stopping_delta = 0.02  # 2%

max_description_embeddings = 600
embedding_dim = 300  # +1 for flag

#downloaded_embedding_file = data_folder + "/word_embeddings/glove.6B.300d.txt"


conv_filters_num = 1024
conv_filters_sizes = [1, 2, 3]
dense_layers = [5000, 2500]
dropout = 0.2

heatmap_threshold = 0.49

In [51]:
# Function for class weights

import numpy as np
from sklearn.utils.class_weight import compute_class_weight
from sklearn.preprocessing import MultiLabelBinarizer


def generate_class_weights(class_series, multi_class=True, one_hot_encoded=False):
  """
  Method to generate class weights given a set of multi-class or multi-label labels, both one-hot-encoded or not.
  Some examples of different formats of class_series and their outputs are:
    - generate_class_weights(['mango', 'lemon', 'banana', 'mango'], multi_class=True, one_hot_encoded=False)
    {'banana': 1.3333333333333333, 'lemon': 1.3333333333333333, 'mango': 0.6666666666666666}
    - generate_class_weights([[1, 0, 0], [0, 1, 0], [0, 0, 1], [1, 0, 0]], multi_class=True, one_hot_encoded=True)
    {0: 0.6666666666666666, 1: 1.3333333333333333, 2: 1.3333333333333333}
    - generate_class_weights([['mango', 'lemon'], ['mango'], ['lemon', 'banana'], ['lemon']], multi_class=False, one_hot_encoded=False)
    {'banana': 1.3333333333333333, 'lemon': 0.4444444444444444, 'mango': 0.6666666666666666}
    - generate_class_weights([[0, 1, 1], [0, 0, 1], [1, 1, 0], [0, 1, 0]], multi_class=False, one_hot_encoded=True)
    {0: 1.3333333333333333, 1: 0.4444444444444444, 2: 0.6666666666666666}
  The output is a dictionary in the format { class_label: class_weight }. In case the input is one hot encoded, the class_label would be index
  of appareance of the label when the dataset was processed. 
  In multi_class this is np.unique(class_series) and in multi-label np.unique(np.concatenate(class_series)).
  Author: Angel Igareta (angel@igareta.com)
  """
  if multi_class:
    # If class is one hot encoded, transform to categorical labels to use compute_class_weight   
    if one_hot_encoded:
      class_series = np.argmax(class_series, axis=1)
  
    # Compute class weights with sklearn method
    class_labels = np.unique(class_series)
    class_weights = compute_class_weight(class_weight='balanced', classes=class_labels, y=class_series)

    print(class_series)
    print(class_labels)

    return dict(zip(class_labels, class_weights))
  else:
    # It is neccessary that the multi-label values are one-hot encoded
    mlb = None
    if not one_hot_encoded:
      mlb = MultiLabelBinarizer()
      class_series = mlb.fit_transform(class_series)

    n_samples = len(class_series)
    n_classes = len(class_series[0])
    print(n_samples)
    print(n_classes)

    # Count each class frequency
    class_count = [0] * n_classes
    for classes in class_series:
        for index in range(n_classes):
            if classes[index] != 0:
                class_count[index] += 1
    
    # Compute class weights using balanced method
    class_weights = [n_samples / (n_classes * freq) if freq > 0 else 1 for freq in class_count]
    class_labels = range(len(class_weights)) if mlb is None else mlb.classes_
    return dict(zip(class_labels, class_weights))
    #return class_weights

In [52]:
class_series = np.array(train_labels)
class_wt = generate_class_weights(class_series, multi_class=False, one_hot_encoded=True)
print(class_wt)
#class_wt = torch.tensor(class_wt)
#print(class_wt)

16197
9
{0: 0.7289050897799378, 1: 0.7318693235732683, 2: 1.229280510018215, 3: 1.1095355528154542, 4: 0.21877785882162248, 5: 0.7523690078037905, 6: 33.32716049382716, 7: 56.239583333333336, 8: 6.4504181600955794}


In [53]:
def model_multiconv_1d(num_permissions):
    #embedding_dim = embedding_dim
    sequence_length = max_description_embeddings

    input_layer = Input(shape=(None,))

    conv_layers = []
    for filter_size in conv_filters_sizes:
        conv_layer_i = Embedding(num_words,
                                 output_dim=embedding_dim,
                                 input_length=sequence_length,
                                 weights=[embedding_matrix],
                                 trainable=False)(input_layer)
        conv_layer_i = Conv1D(filters=conv_filters_num,
                              kernel_size=filter_size,
                              padding='same',
                              activation='relu')(conv_layer_i)
        conv_layer_i = GlobalMaxPooling1D()(conv_layer_i)

        conv_layers.append(conv_layer_i)

    if len(conv_layers) == 1:
        previous_layer = conv_layers[0]
    else:
        concatenated_layer = concatenate(conv_layers, axis=-1)
        previous_layer = concatenated_layer

    for n_neurons in dense_layers:
        previous_layer = Dense(n_neurons, activation='relu')(previous_layer)
        previous_layer = Dropout(dropout)(previous_layer)

    output_layer = Dense(num_permissions, activation='sigmoid')(previous_layer)

    return keras.Model(inputs=input_layer, outputs=output_layer)

In [54]:
model = model_multiconv_1d(num_permissions)

In [55]:
model.compile(loss="binary_crossentropy", optimizer=Adam(0.0001), metrics=['accuracy'])
              
#metrics=[metrics.fb_micro, metrics.fb_macro, metrics.precision, metrics.recall])
train_metric = 'val_fb_macro'

In [56]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, None)]       0           []                               
                                                                                                  
 embedding (Embedding)          (None, None, 300)    19935000    ['input_1[0][0]']                
                                                                                                  
 embedding_1 (Embedding)        (None, None, 300)    19935000    ['input_1[0][0]']                
                                                                                                  
 embedding_2 (Embedding)        (None, None, 300)    19935000    ['input_1[0][0]']                
                                                                                              

In [57]:
checkpoint = ModelCheckpoint('weights_cnn_sentece.hdf5', monitor='val_acc', verbose=1, save_best_only=True, mode='auto')

model.fit(train_padded, train_labels, 
          batch_size=batch_size, epochs=10, class_weight=class_wt,
          verbose=1, callbacks=[checkpoint], validation_data=(validation_padded, validation_labels))

Epoch 1/10
507/507 [==============================] - 112s 191ms/step - loss: 0.1927 - accuracy: 0.2862 - val_loss: 0.2518 - val_accuracy: 0.2657
Epoch 2/10
507/507 [==============================] - 96s 189ms/step - loss: 0.1530 - accuracy: 0.2927 - val_loss: 0.2730 - val_accuracy: 0.3000
Epoch 3/10
507/507 [==============================] - 96s 189ms/step - loss: 0.1334 - accuracy: 0.3022 - val_loss: 0.2280 - val_accuracy: 0.2647
Epoch 4/10
507/507 [==============================] - 96s 189ms/step - loss: 0.1143 - accuracy: 0.2999 - val_loss: 0.2306 - val_accuracy: 0.2785
Epoch 5/10
507/507 [==============================] - 95s 188ms/step - loss: 0.0917 - accuracy: 0.3069 - val_loss: 0.2241 - val_accuracy: 0.3175
Epoch 6/10
507/507 [==============================] - 95s 188ms/step - loss: 0.0812 - accuracy: 0.3236 - val_loss: 0.2245 - val_accuracy: 0.2849
Epoch 7/10
507/507 [==============================] - 95s 188ms/step - loss: 0.0679 - accuracy: 0.3362 - val_loss: 0.2373 - val_a

## 8. Performance on Test Set

In [58]:
test_labels = df_test[target_list]
test_input = df_test["Clean_Description"]

In [59]:
test_sequences = tokenizer.texts_to_sequences(test_input)
test_padded = pad_sequences(
    test_sequences, maxlen=MAX_SEQUENCE_LENGTH, padding="post", truncating="post"
)

In [60]:
print(df_test.Clean_Description[0])
print(test_sequences[0])

poetry lover find best poetry poets urdu problem solved app app contains time best urdu poetry famous urdu poets offline urdu poetry collection best urdu poetry famous urdu poets like mohsin naqvi mirza ghalib saghir siddiqui others also contains romantic urdu poetry sad urdu poetry urdu ghazals offline urdu shayari urdu shayari app contains best ever poetry famous poets times app contains best variety urdu poetry categories romantic poetry sad poetry attitude poetry ghazals inspirational poetry main features offline poetry collection gallery view poetry collection zooming feature save share poetry images app contains offline poetry urdu ghazals urdu poetry poetry famous poets urdu shayari much more
[1948, 1809, 28, 23, 1948, 9605, 394, 739, 2156, 1, 1, 208, 13, 23, 394, 1948, 1162, 394, 9605, 115, 394, 1948, 164, 23, 394, 1948, 1162, 394, 9605, 19, 19879, 413, 16, 208, 462, 394, 1948, 1739, 394, 1948, 394, 10742, 115, 394, 778, 394, 778, 1, 208, 23, 519, 1948, 1162, 9605, 211, 1, 208,

In [61]:
print(test_labels)

      Camera  Location  Microphone  Contacts  Storage  Phone  SMS  Call_Log  \
0          0         0           0         0        1      0    0         0   
1          0         0           0         0        1      0    0         0   
2          0         0           0         0        1      0    0         0   
3          0         0           0         0        0      0    0         0   
4          1         0           0         0        0      1    0         0   
...      ...       ...         ...       ...      ...    ...  ...       ...   
4619       0         1           0         0        0      0    0         0   
4620       0         0           0         0        0      0    0         0   
4621       0         0           0         0        1      0    0         0   
4622       0         1           1         0        1      0    0         0   
4623       0         0           0         0        0      0    0         0   

      Calendar  
0            0  
1            0  


In [62]:
print(test_padded.shape)
print(test_labels.shape)

(4624, 600)
(4624, 9)


In [63]:
# Generate predictions (probabilities -- the output of the last layer)
# on new data using `predict`

predictions = model.predict(test_padded)
print("predictions shape:", predictions.shape)

predictions shape: (4624, 9)


In [64]:
predictions[0:10]

array([[3.1980205e-02, 1.3551162e-02, 4.5370300e-05, 8.5997791e-04,
        3.7329775e-01, 6.1892378e-03, 7.0297232e-05, 3.1071788e-06,
        7.9987483e-05],
       [1.1702943e-04, 4.6422699e-04, 2.6757030e-05, 1.7782768e-04,
        1.4687860e-03, 2.4022404e-03, 3.4208692e-04, 6.1635233e-06,
        2.1519550e-04],
       [8.8439137e-01, 7.6314288e-01, 8.4695362e-02, 1.9067758e-01,
        9.8659652e-01, 2.8356037e-01, 2.9665851e-03, 6.7384425e-04,
        2.3735132e-02],
       [2.0332490e-03, 4.9971721e-01, 5.5152355e-05, 1.3477807e-02,
        5.2478307e-01, 1.8742912e-01, 3.8512421e-04, 4.7701429e-05,
        1.9888273e-03],
       [1.7391384e-02, 1.2286594e-02, 4.4313932e-05, 2.2243157e-01,
        1.7792411e-01, 9.2840232e-02, 1.0061099e-02, 6.1086618e-04,
        2.2180660e-03],
       [1.8473348e-01, 8.5689530e-02, 4.2773532e-03, 6.8228289e-02,
        9.9729413e-01, 4.1089132e-02, 2.0228350e-04, 2.0228877e-05,
        2.3958381e-04],
       [8.8311398e-01, 9.4529122e-01, 8.

In [65]:
true_labels = test_labels.to_numpy()
true_labels[0:10]

array([[0, 0, 0, 0, 1, 0, 0, 0, 0],
       [0, 0, 0, 0, 1, 0, 0, 0, 0],
       [0, 0, 0, 0, 1, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0],
       [1, 0, 0, 0, 0, 1, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 1, 1, 1, 1, 0, 1, 0],
       [0, 0, 0, 1, 1, 0, 0, 0, 0],
       [1, 1, 0, 0, 1, 1, 0, 0, 0],
       [0, 0, 0, 1, 1, 1, 0, 0, 0]])

In [66]:
np.save("predictions.npy", predictions)
loaded_predictions = np.load("predictions.npy")
print(loaded_predictions)

[[3.1980205e-02 1.3551162e-02 4.5370300e-05 ... 7.0297232e-05
  3.1071788e-06 7.9987483e-05]
 [1.1702943e-04 4.6422699e-04 2.6757030e-05 ... 3.4208692e-04
  6.1635233e-06 2.1519550e-04]
 [8.8439137e-01 7.6314288e-01 8.4695362e-02 ... 2.9665851e-03
  6.7384425e-04 2.3735132e-02]
 ...
 [9.7970974e-01 8.4647083e-01 3.8862246e-01 ... 1.2918990e-02
  6.4943773e-03 8.1876293e-02]
 [5.6357356e-03 9.9879336e-01 9.9877948e-01 ... 1.8649130e-08
  5.8128222e-09 8.0217497e-04]
 [1.6288692e-03 2.2953430e-03 2.4904939e-04 ... 1.9487352e-04
  9.7763104e-06 2.0285018e-04]]


In [67]:
np.save("true_labels.npy", true_labels)
loaded_true_labels = np.load("true_labels.npy")
print(loaded_true_labels)

[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 1 1 ... 0 0 0]
 [0 0 0 ... 0 0 0]]


## 9. Threshold Calculation

In [68]:
# Import module for data manipulation
import pandas as pd
# Import module for linear algebra
import numpy as np
# Import module for data simulation
from sklearn.datasets import make_classification     # Create a synthetic dataframe
from sklearn.linear_model import LogisticRegression  # Classification model
from sklearn.model_selection import train_test_split # Split the dataframe
from sklearn.metrics import roc_curve                # Calculate the ROC curve
from sklearn.metrics import precision_recall_curve   # Calculate the Precision-Recall curve
from sklearn.metrics import f1_score                 # Calculate the F-score
# Import module for data visualization
from plotnine import *
import plotnine

In [69]:
thresh_f = [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
thresh_roc = [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]

In [70]:
n = 9
for i in range(0, n):
  print('permission ',i)

  predictions = np.load("predictions.npy")
  true_labels = np.load("true_labels.npy")

  # Array for finding the optimal threshold
  thresholds = np.arange(0.0, 1.0, 0.0001)
  fscore = np.zeros(shape=(len(thresholds)))
  print('Length of sequence: {}'.format(len(thresholds)))

  labels = true_labels[:, i]
  pred = predictions[:, i]

  # Fit the model
  for index, elem in enumerate(thresholds):
    # Corrected probabilities
    y_pred_prob = (pred > elem).astype('int')
    # Calculate the f-score
    fscore[index] = f1_score(labels, y_pred_prob)

  # Find the optimal threshold
  index = np.argmax(fscore)
  thresholdOpt = round(thresholds[index], ndigits = 4)
  fscoreOpt = round(fscore[index], ndigits = 4)
  thresh_f[i] = thresholdOpt
  print('Best Threshold: {} with F-Score: {}'.format(thresholdOpt, fscoreOpt))

print("-------------------------------------")
print("optimal threshold tuning for f-score")
print(thresh_f)

permission  0
Length of sequence: 10000
Best Threshold: 0.7374 with F-Score: 0.6878
permission  1
Length of sequence: 10000
Best Threshold: 0.7021 with F-Score: 0.5832
permission  2
Length of sequence: 10000
Best Threshold: 0.1403 with F-Score: 0.5661
permission  3
Length of sequence: 10000
Best Threshold: 0.4267 with F-Score: 0.5417
permission  4
Length of sequence: 10000
Best Threshold: 0.3503 with F-Score: 0.8026
permission  5
Length of sequence: 10000
Best Threshold: 0.3802 with F-Score: 0.5081
permission  6
Length of sequence: 10000
Best Threshold: 0.0678 with F-Score: 0.1622
permission  7
Length of sequence: 10000
Best Threshold: 0.2605 with F-Score: 0.2857
permission  8
Length of sequence: 10000
Best Threshold: 0.1163 with F-Score: 0.3556
-------------------------------------
optimal threshold tuning for f-score
[0.7374, 0.7021, 0.1403, 0.4267, 0.3503, 0.3802, 0.0678, 0.2605, 0.1163]


In [71]:
##for roc curve with g-mean

n = 9
for i in range(0, n):
  print('permission ',i)

  predictions = np.load("predictions.npy")
  true_labels = np.load("true_labels.npy")

  labels = true_labels[:, i]
  pred = predictions[:, i]

  # Create the ROC curve
  fpr, tpr, thresholds = roc_curve(labels, pred)

  df_fpr_tpr = pd.DataFrame({'FPR':fpr, 'TPR':tpr, 'Threshold':thresholds})

  # Calculate the G-mean
  gmean = np.sqrt(tpr * (1 - fpr))

  # Find the optimal threshold
  index = np.argmax(gmean)
  thresholdOpt = round(thresholds[index], ndigits = 4)
  gmeanOpt = round(gmean[index], ndigits = 4)
  fprOpt = round(fpr[index], ndigits = 4)
  tprOpt = round(tpr[index], ndigits = 4)

  thresh_roc[i] = thresholdOpt
  print('Best Threshold: {} with G-Mean: {}'.format(thresholdOpt, gmeanOpt))
  print('FPR: {}, TPR: {}'.format(fprOpt, tprOpt))

print("-------------------------------------")
print("ROC curve with G-mean threshold tuning")
print(thresh_roc)

permission  0
Best Threshold: 0.2101999968290329 with G-Mean: 0.8487
FPR: 0.1428, TPR: 0.8403
permission  1
Best Threshold: 0.25839999318122864 with G-Mean: 0.7798
FPR: 0.2011, TPR: 0.7612
permission  2
Best Threshold: 0.004600000102072954 with G-Mean: 0.8273
FPR: 0.2085, TPR: 0.8647
permission  3
Best Threshold: 0.053700000047683716 with G-Mean: 0.8045
FPR: 0.1991, TPR: 0.808
permission  4
Best Threshold: 0.7124999761581421 with G-Mean: 0.7924
FPR: 0.1908, TPR: 0.776
permission  5
Best Threshold: 0.08169999718666077 with G-Mean: 0.766
FPR: 0.2455, TPR: 0.7776
permission  6
Best Threshold: 0.011599999852478504 with G-Mean: 0.8768
FPR: 0.0605, TPR: 0.8182
permission  7
Best Threshold: 0.0008999999845400453 with G-Mean: 0.9328
FPR: 0.1299, TPR: 1.0
permission  8
Best Threshold: 0.013799999840557575 with G-Mean: 0.8388
FPR: 0.1442, TPR: 0.8222
-------------------------------------
ROC curve with G-mean threshold tuning
[0.2102, 0.2584, 0.0046, 0.0537, 0.7125, 0.0817, 0.0116, 0.0009, 0.013

## 10. Performance Score

In [ ]:
#Fscore micro for different thresholds-

In [72]:
#predictions = np.load("predictions.npy")
predictions = np.load("predictions.npy")
true_labels = np.load("true_labels.npy")

In [73]:
# Function to calculate the accuracy of our predictions vs labels

import numpy as np
from sklearn.metrics import f1_score

def f_at_1(preds, labels):
    #print('my_print_2')
    acc = [0, 0 ,0 ,0 ,0 ,0, 0, 0, 0]
    preds_th = preds
    
    preds_th[:, 0] = np.array(preds[:, 0]) >= thresh_f[0]
    preds_th[:, 1] = np.array(preds[:, 1]) >= thresh_f[1]
    preds_th[:, 2] = np.array(preds[:, 2]) >= thresh_f[2]
    preds_th[:, 3] = np.array(preds[:, 3]) >= thresh_f[3]
    preds_th[:, 4] = np.array(preds[:, 4]) >= thresh_f[4]
    preds_th[:, 5] = np.array(preds[:, 5]) >= thresh_f[5]
    preds_th[:, 6] = np.array(preds[:, 6]) >= thresh_f[6]
    preds_th[:, 7] = np.array(preds[:, 7]) >= thresh_f[7]
    preds_th[:, 8] = np.array(preds[:, 8]) >= thresh_f[8]

   
    acc[0] = f1_score(labels[:, 0], preds_th[:, 0])
    acc[1] = f1_score(labels[:, 1], preds_th[:, 1])
    acc[2] = f1_score(labels[:, 2], preds_th[:, 2])
    acc[3] = f1_score(labels[:, 3], preds_th[:, 3])
    acc[4] = f1_score(labels[:, 4], preds_th[:, 4])
    acc[5] = f1_score(labels[:, 5], preds_th[:, 5])
    acc[6] = f1_score(labels[:, 6], preds_th[:, 6])
    acc[7] = f1_score(labels[:, 7], preds_th[:, 7])
    acc[8] = f1_score(labels[:, 8], preds_th[:, 8])

   #f1_score_micro = metrics.f1_score(targets, outputs, average='micro')

    #print(acc)
    return acc

In [74]:
eval_accuracy = f_at_1(predictions, true_labels)

np.save("F1_CV4_N20k_TCNN.npy", eval_accuracy)

# Report the final accuracy for this validation run.
print("  Camera    : {0:.4f}".format(eval_accuracy[0]))
print("  Location  : {0:.4f}".format(eval_accuracy[1]))
print("  Microphone: {0:.4f}".format(eval_accuracy[2]))
print("  Contacts  : {0:.4f}".format(eval_accuracy[3]))
print("  Storage   : {0:.4f}".format(eval_accuracy[4]))
print("  Phone     : {0:.4f}".format(eval_accuracy[5]))
print("  SMS       : {0:.4f}".format(eval_accuracy[6]))
print("  Call_Log  : {0:.4f}".format(eval_accuracy[7]))
print("  Calendar  : {0:.4f}".format(eval_accuracy[8]))

print("")
avg_score = (np.sum(eval_accuracy, dtype = np.float32)) / 9
print("  Average F1 score: {0:.4f}".format(avg_score))

  Camera    : 0.6878
  Location  : 0.5832
  Microphone: 0.5661
  Contacts  : 0.5417
  Storage   : 0.8026
  Phone     : 0.5081
  SMS       : 0.1622
  Call_Log  : 0.2857
  Calendar  : 0.3556

  Average F1 score: 0.4992


In [ ]:
#Fscore micro for different thresholds-

In [75]:
#predictions = np.load("predictions.npy")
predictions = np.load("predictions.npy")
true_labels = np.load("true_labels.npy")

In [76]:
# Function to calculate the accuracy of our predictions vs labels

import numpy as np
from sklearn.metrics import f1_score

def f1micro_accuracy(preds, labels):
    #print('my_print_2')
    acc = [0, 0 ,0 ,0 ,0 ,0, 0, 0, 0]
    preds_th = preds
    
    preds_th[:, 0] = np.array(preds[:, 0]) >= thresh_f[0]
    preds_th[:, 1] = np.array(preds[:, 1]) >= thresh_f[1]
    preds_th[:, 2] = np.array(preds[:, 2]) >= thresh_f[2]
    preds_th[:, 3] = np.array(preds[:, 3]) >= thresh_f[3]
    preds_th[:, 4] = np.array(preds[:, 4]) >= thresh_f[4]
    preds_th[:, 5] = np.array(preds[:, 5]) >= thresh_f[5]
    preds_th[:, 6] = np.array(preds[:, 6]) >= thresh_f[6]
    preds_th[:, 7] = np.array(preds[:, 7]) >= thresh_f[7]
    preds_th[:, 8] = np.array(preds[:, 8]) >= thresh_f[8]

    acc[0] = f1_score(labels[:, 0], preds_th[:, 0], average='micro')
    acc[1] = f1_score(labels[:, 1], preds_th[:, 1], average='micro')
    acc[2] = f1_score(labels[:, 2], preds_th[:, 2], average='micro')
    acc[3] = f1_score(labels[:, 3], preds_th[:, 3], average='micro')
    acc[4] = f1_score(labels[:, 4], preds_th[:, 4], average='micro')
    acc[5] = f1_score(labels[:, 5], preds_th[:, 5], average='micro')
    acc[6] = f1_score(labels[:, 6], preds_th[:, 6], average='micro')
    acc[7] = f1_score(labels[:, 7], preds_th[:, 7], average='micro')
    acc[8] = f1_score(labels[:, 8], preds_th[:, 8], average='micro')
    
   #f1_score_micro = metrics.f1_score(targets, outputs, average='micro')

    #print(acc)
    return acc

In [77]:
eval_accuracy = f1micro_accuracy(predictions, true_labels)

np.save("F1Mic_CV4_N20k_TCNN.npy", eval_accuracy)

# Report the final accuracy for this validation run.

print("  Camera    : {0:.4f}".format(eval_accuracy[0]))
print("  Location  : {0:.4f}".format(eval_accuracy[1]))
print("  Microphone: {0:.4f}".format(eval_accuracy[2]))
print("  Contacts  : {0:.4f}".format(eval_accuracy[3]))
print("  Storage   : {0:.4f}".format(eval_accuracy[4]))
print("  Phone     : {0:.4f}".format(eval_accuracy[5]))
print("  SMS       : {0:.4f}".format(eval_accuracy[6]))
print("  Call_Log  : {0:.4f}".format(eval_accuracy[7]))
print("  Calendar  : {0:.4f}".format(eval_accuracy[8]))

print("")

avg_score = (np.sum(eval_accuracy, dtype = np.float32)) / 9
print("  Average F1 (micro) score: {0:.4f}".format(avg_score))

  Camera    : 0.9001
  Location  : 0.8646
  Microphone: 0.9148
  Contacts  : 0.8953
  Storage   : 0.7820
  Phone     : 0.8547
  SMS       : 0.9933
  Call_Log  : 0.9989
  Calendar  : 0.9749

  Average F1 (micro) score: 0.9087


In [ ]:
#roc-auc score for different thresholds-

In [78]:
import numpy as np
predictions = np.load("predictions.npy")
true_labels = np.load("true_labels.npy")

In [79]:
# Function to calculate the accuracy of our predictions vs labels

import numpy as np
from sklearn.metrics import roc_auc_score

def roc_auc(preds, labels):
    #print('my_print_2')
    acc = [0, 0 ,0 ,0 ,0 ,0, 0, 0, 0]
    preds_th = preds
    
    preds_th[:, 0] = np.array(preds[:, 0]) >= thresh_roc[0]
    preds_th[:, 1] = np.array(preds[:, 1]) >= thresh_roc[1]
    preds_th[:, 2] = np.array(preds[:, 2]) >= thresh_roc[2]
    preds_th[:, 3] = np.array(preds[:, 3]) >= thresh_roc[3]
    preds_th[:, 4] = np.array(preds[:, 4]) >= thresh_roc[4]
    preds_th[:, 5] = np.array(preds[:, 5]) >= thresh_roc[5]
    preds_th[:, 6] = np.array(preds[:, 6]) >= thresh_roc[6]
    preds_th[:, 7] = np.array(preds[:, 7]) >= thresh_roc[7]
    preds_th[:, 8] = np.array(preds[:, 8]) >= thresh_roc[8]


    acc[0] = roc_auc_score(labels[:, 0], preds_th[:, 0])
    acc[1] = roc_auc_score(labels[:, 1], preds_th[:, 1])
    acc[2] = roc_auc_score(labels[:, 2], preds_th[:, 2])
    acc[3] = roc_auc_score(labels[:, 3], preds_th[:, 3])
    acc[4] = roc_auc_score(labels[:, 4], preds_th[:, 4])
    acc[5] = roc_auc_score(labels[:, 5], preds_th[:, 5])
    acc[6] = roc_auc_score(labels[:, 6], preds_th[:, 6])
    acc[7] = roc_auc_score(labels[:, 7], preds_th[:, 7])
    acc[8] = roc_auc_score(labels[:, 8], preds_th[:, 8])
   

    #print(acc)
    return acc

In [80]:
#roc-auc score

eval_accuracy = roc_auc(predictions, true_labels)

np.save("ROC_CV4_N20k_TCNN.npy", eval_accuracy)

# Report the final accuracy for this validation run.

print("  Camera    : {0:.4f}".format(eval_accuracy[0]))
print("  Location  : {0:.4f}".format(eval_accuracy[1]))
print("  Microphone: {0:.4f}".format(eval_accuracy[2]))
print("  Contacts  : {0:.4f}".format(eval_accuracy[3]))
print("  Storage   : {0:.4f}".format(eval_accuracy[4]))
print("  Phone     : {0:.4f}".format(eval_accuracy[5]))
print("  SMS       : {0:.4f}".format(eval_accuracy[6]))
print("  Call_Log  : {0:.4f}".format(eval_accuracy[7]))
print("  Calendar  : {0:.4f}".format(eval_accuracy[8]))

print("")

avg_score = (np.sum(eval_accuracy, dtype = np.float32)) / 9
print("  Average ROC_AUC score: {0:.4f}".format(avg_score))

  Camera    : 0.8481
  Location  : 0.7793
  Microphone: 0.8279
  Contacts  : 0.8043
  Storage   : 0.7924
  Phone     : 0.7653
  SMS       : 0.8336
  Call_Log  : 0.9331
  Calendar  : 0.8387

  Average ROC_AUC score: 0.8247


# ACNET performance

#### a) Data Preparation

In [81]:
## drive path
acnet_path = "/content/drive/MyDrive/MetadataCSV/acnet_dataset_preprocess.csv"

In [82]:
df_acnet = pd.read_csv(acnet_path) 
print(df_acnet.shape)

(1417, 14)


In [83]:
df_acnet = df_acnet.dropna(subset=['Clean_Description'])

In [84]:
df_acnet.head()

,app_id,description,Storage,Contacts,Location,Camera,Microphone,SMS,Call_Log,Phone,Calendar,Settings,Tasks,Clean_Description
0,0,ROOT is REQUIRED for automatic synchronization...,1,0,1,0,0,0,0,0,0,0,0,root is required for automatic synchronization...
1,1,This app delivers short scriptures containing ...,0,0,0,0,0,1,0,0,0,0,0,this app delivers short scriptures containing ...
2,2,This game is surprisingly simple and very addi...,0,0,0,0,0,0,0,0,0,0,0,this game is surprisingly simple and very addi...
3,3,It is an online RPG game based on LBS location...,0,0,1,0,0,0,0,0,0,0,0,it is an online rpg game based on lbs location...
4,4,Christmas is in the air. Get yourself in the h...,0,1,0,0,0,0,0,0,0,1,1,christmas is in the air. get yourself in the h...


In [85]:
df_acnet["Clean_Description"] = df_acnet["Clean_Description"].map(remove_stopwords)

In [86]:
df_acnet["Clean_Description"] = df_acnet["Clean_Description"].map(lambda x: remove_punct(x))

In [87]:
acnet_labels = df_acnet[target_list]
acnet_input = df_acnet["Clean_Description"]

In [88]:
acnet_sequences = tokenizer.texts_to_sequences(acnet_input)
acnet_padded = pad_sequences(
    acnet_sequences, maxlen=MAX_SEQUENCE_LENGTH, padding="post", truncating="post"
)

In [89]:
print(acnet_padded.shape)
print(acnet_labels.shape)

(1414, 600)
(1414, 9)


#### b) Get Predictions

In [90]:
# Generate predictions (probabilities -- the output of the last layer)
# on new data using `predict`

predictions = model.predict(acnet_padded)
print("predictions shape:", predictions.shape)

predictions shape: (1414, 9)


In [91]:
acnet_labels = acnet_labels.to_numpy()

In [92]:
np.save("acnet_predictions.npy", predictions)
loaded_predictions = np.load("acnet_predictions.npy")

np.save("acnet_labels.npy", acnet_labels)
loaded_true_labels = np.load("acnet_labels.npy")

#### c) Threshold Calculation

In [93]:
thresh_f = [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
thresh_roc = [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]

In [94]:
n = 9
for i in range(0, n):
  print('permission ',i)

  predictions = np.load("acnet_predictions.npy")
  true_labels = np.load("acnet_labels.npy")

  # Array for finding the optimal threshold
  thresholds = np.arange(0.0, 1.0, 0.0001)
  fscore = np.zeros(shape=(len(thresholds)))
  print('Length of sequence: {}'.format(len(thresholds)))

  labels = true_labels[:, i]
  pred = predictions[:, i]

  # Fit the model
  for index, elem in enumerate(thresholds):
    # Corrected probabilities
    y_pred_prob = (pred > elem).astype('int')
    # Calculate the f-score
    fscore[index] = f1_score(labels, y_pred_prob)

  # Find the optimal threshold
  index = np.argmax(fscore)
  thresholdOpt = round(thresholds[index], ndigits = 4)
  fscoreOpt = round(fscore[index], ndigits = 4)
  thresh_f[i] = thresholdOpt
  print('Best Threshold: {} with F-Score: {}'.format(thresholdOpt, fscoreOpt))

print("-------------------------------------")
print("optimal threshold tuning for f-score")
print(thresh_f)

permission  0
Length of sequence: 10000
Best Threshold: 0.4498 with F-Score: 0.4746
permission  1
Length of sequence: 10000
Best Threshold: 0.529 with F-Score: 0.5956
permission  2
Length of sequence: 10000
Best Threshold: 0.273 with F-Score: 0.5552
permission  3
Length of sequence: 10000
Best Threshold: 0.1097 with F-Score: 0.6127
permission  4
Length of sequence: 10000
Best Threshold: 0.0192 with F-Score: 0.5895
permission  5
Length of sequence: 10000
Best Threshold: 0.6522 with F-Score: 0.3763
permission  6
Length of sequence: 10000
Best Threshold: 0.0146 with F-Score: 0.5747
permission  7
Length of sequence: 10000
Best Threshold: 0.0034 with F-Score: 0.3438
permission  8
Length of sequence: 10000
Best Threshold: 0.0988 with F-Score: 0.5826
-------------------------------------
optimal threshold tuning for f-score
[0.4498, 0.529, 0.273, 0.1097, 0.0192, 0.6522, 0.0146, 0.0034, 0.0988]


In [95]:
##for roc curve with g-mean

n = 9
for i in range(0, n):
  print('permission ',i)

  predictions = np.load("acnet_predictions.npy")
  true_labels = np.load("acnet_labels.npy")

  labels = true_labels[:, i]
  pred = predictions[:, i]

  # Create the ROC curve
  fpr, tpr, thresholds = roc_curve(labels, pred)

  df_fpr_tpr = pd.DataFrame({'FPR':fpr, 'TPR':tpr, 'Threshold':thresholds})

  # Calculate the G-mean
  gmean = np.sqrt(tpr * (1 - fpr))

  # Find the optimal threshold
  index = np.argmax(gmean)
  thresholdOpt = round(thresholds[index], ndigits = 4)
  gmeanOpt = round(gmean[index], ndigits = 4)
  fprOpt = round(fpr[index], ndigits = 4)
  tprOpt = round(tpr[index], ndigits = 4)

  thresh_roc[i] = thresholdOpt
  print('Best Threshold: {} with G-Mean: {}'.format(thresholdOpt, gmeanOpt))
  print('FPR: {}, TPR: {}'.format(fprOpt, tprOpt))

print("-------------------------------------")
print("ROC curve with G-mean threshold tuning")
print(thresh_roc)

permission  0
Best Threshold: 0.15389999747276306 with G-Mean: 0.7339
FPR: 0.3333, TPR: 0.8079
permission  1
Best Threshold: 0.47380000352859497 with G-Mean: 0.763
FPR: 0.2099, TPR: 0.7368
permission  2
Best Threshold: 0.020400000736117363 with G-Mean: 0.763
FPR: 0.1872, TPR: 0.7162
permission  3
Best Threshold: 0.11110000312328339 with G-Mean: 0.7193
FPR: 0.2472, TPR: 0.6874
permission  4
Best Threshold: 0.7087000012397766 with G-Mean: 0.572
FPR: 0.4026, TPR: 0.5477
permission  5
Best Threshold: 0.1890999972820282 with G-Mean: 0.7368
FPR: 0.3123, TPR: 0.7895
permission  6
Best Threshold: 0.00430000014603138 with G-Mean: 0.7939
FPR: 0.2321, TPR: 0.8208
permission  7
Best Threshold: 0.0007999999797903001 with G-Mean: 0.7594
FPR: 0.2618, TPR: 0.7812
permission  8
Best Threshold: 0.04610000178217888 with G-Mean: 0.8705
FPR: 0.1145, TPR: 0.8558
-------------------------------------
ROC curve with G-mean threshold tuning
[0.1539, 0.4738, 0.0204, 0.1111, 0.7087, 0.1891, 0.0043, 0.0008, 0.046

#### d) Accuracy Score

In [ ]:
#Fscore micro for different thresholds-

In [96]:
#predictions = np.load("predictions.npy")
predictions = np.load("acnet_predictions.npy")
true_labels = np.load("acnet_labels.npy")

In [97]:
eval_accuracy = f_at_1(predictions, true_labels)

np.save("ACNET_F1_CV4_N20k_TCNN.npy", eval_accuracy)

# Report the final accuracy for this validation run.
print("  Camera    : {0:.4f}".format(eval_accuracy[0]))
print("  Location  : {0:.4f}".format(eval_accuracy[1]))
print("  Microphone: {0:.4f}".format(eval_accuracy[2]))
print("  Contacts  : {0:.4f}".format(eval_accuracy[3]))
print("  Storage   : {0:.4f}".format(eval_accuracy[4]))
print("  Phone     : {0:.4f}".format(eval_accuracy[5]))
print("  SMS       : {0:.4f}".format(eval_accuracy[6]))
print("  Call_Log  : {0:.4f}".format(eval_accuracy[7]))
print("  Calendar  : {0:.4f}".format(eval_accuracy[8]))

print("")
avg_score = (np.sum(eval_accuracy, dtype = np.float32)) / 9
print("  Average F1 score: {0:.4f}".format(avg_score))

  Camera    : 0.4746
  Location  : 0.5956
  Microphone: 0.5552
  Contacts  : 0.6127
  Storage   : 0.5895
  Phone     : 0.3763
  SMS       : 0.5747
  Call_Log  : 0.3438
  Calendar  : 0.5826

  Average F1 score: 0.5228


In [ ]:
#Fscore micro for different thresholds-

In [98]:
#predictions = np.load("predictions.npy")
predictions = np.load("acnet_predictions.npy")
true_labels = np.load("acnet_labels.npy")

In [99]:
eval_accuracy = f1micro_accuracy(predictions, true_labels)

np.save("ACNET_F1Mic_CV4_N20k_TCNN.npy", eval_accuracy)

# Report the final accuracy for this validation run.

print("  Camera    : {0:.4f}".format(eval_accuracy[0]))
print("  Location  : {0:.4f}".format(eval_accuracy[1]))
print("  Microphone: {0:.4f}".format(eval_accuracy[2]))
print("  Contacts  : {0:.4f}".format(eval_accuracy[3]))
print("  Storage   : {0:.4f}".format(eval_accuracy[4]))
print("  Phone     : {0:.4f}".format(eval_accuracy[5]))
print("  SMS       : {0:.4f}".format(eval_accuracy[6]))
print("  Call_Log  : {0:.4f}".format(eval_accuracy[7]))
print("  Calendar  : {0:.4f}".format(eval_accuracy[8]))

print("")

avg_score = (np.sum(eval_accuracy, dtype = np.float32)) / 9
print("  Average F1 (micro) score: {0:.4f}".format(avg_score))

  Camera    : 0.7808
  Location  : 0.7935
  Microphone: 0.9116
  Contacts  : 0.7327
  Storage   : 0.4682
  Phone     : 0.8734
  SMS       : 0.8670
  Call_Log  : 0.8515
  Calendar  : 0.9321

  Average F1 (micro) score: 0.8012


In [ ]:
#roc-auc score for different thresholds-

In [100]:
import numpy as np
predictions = np.load("acnet_predictions.npy")
true_labels = np.load("acnet_labels.npy")

In [101]:
#roc-auc score

eval_accuracy = roc_auc(predictions, true_labels)

np.save("ACNET_ROC_CV4_N20k_TCNN.npy", eval_accuracy)

# Report the final accuracy for this validation run.

print("  Camera    : {0:.4f}".format(eval_accuracy[0]))
print("  Location  : {0:.4f}".format(eval_accuracy[1]))
print("  Microphone: {0:.4f}".format(eval_accuracy[2]))
print("  Contacts  : {0:.4f}".format(eval_accuracy[3]))
print("  Storage   : {0:.4f}".format(eval_accuracy[4]))
print("  Phone     : {0:.4f}".format(eval_accuracy[5]))
print("  SMS       : {0:.4f}".format(eval_accuracy[6]))
print("  Call_Log  : {0:.4f}".format(eval_accuracy[7]))
print("  Calendar  : {0:.4f}".format(eval_accuracy[8]))

print("")

avg_score = (np.sum(eval_accuracy, dtype = np.float32)) / 9
print("  Average ROC_AUC score: {0:.4f}".format(avg_score))

  Camera    : 0.7351
  Location  : 0.7618
  Microphone: 0.7645
  Contacts  : 0.7189
  Storage   : 0.5725
  Phone     : 0.7386
  SMS       : 0.7943
  Call_Log  : 0.7571
  Calendar  : 0.8706

  Average ROC_AUC score: 0.7459
